In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
nb_path = '/content/notebooks'
os.symlink('/content/drive/MyDrive/Colab Notebooks', nb_path)
sys.path.insert(0,nb_path)

Mounted at /content/drive


In [ ]:
# !pip install --target=$nb_path tilse
!apt-get install libxml-parser-perl

import datetime
import pprint
import json

from tilse.data import timelines
from tilse.evaluation import rouge

In [ ]:
# !python3 /content/drive/MyDrive/Colab\ Notebooks/bin/get-and-preprocess-data --corpus_name "timeline17"
# os.chdir('/content/drive/MyDrive/Colab Notebooks/news-tls')

# !pip install -r requirements.txt
# !pip install -e .

# !python3 preprocessing/preprocess_tokenize.py --dataset datasets/t17/

# !python3 preprocessing/preprocess_heideltime.py --dataset datasets/t17/ --heideltime '/content/drive/MyDrive/Colab Notebooks/tilse/tools/heideltime'

In [ ]:
import pandas as pd
event_data = pd.read_csv('drive/MyDrive/Colab Notebooks/event_timeline_generation/news-tls data/CWMG/CWMG_letters_timeline_data.csv')

# event_data.isna().sum()
event_data.dropna(subset = ["title"], inplace=True)
# event_data.head()
df_new = event_data.groupby(event_data['date']).agg(lambda x: x.tolist())

event_data.head(15)

,Unnamed: 0,date,event,title
0,0,1930-09-02T00:00:00+00:00,"LETTER TO MOTILAL NEHRU DELHI , January 3 , 1...","LETTER TO MOTILAL NEHRU DELHI , January 3 , 19..."
1,1,1930-01-03T00:00:00+00:00,LETTER TO S. SRINIVASA IYENGAR Your conduct i...,LETTER TO S. SRINIVASA IYENGAR Your conduct in...
3,3,1930-04-11T00:00:00+00:00,"KUNVARJI , Rami and the children arrived here...","KUNVARJI , Rami and the children arrived here ..."
4,4,1930-07-18T00:00:00+00:00,It was not very satisfying . The text of the ...,"The text of the original , which is in Gujarat..."
6,6,1930-11-28T00:00:00+00:00,", Your letter has arrived . Let us be content...",Let us be content with exchange of letters . B...
7,7,1930-04-14T00:00:00+00:00,All that is necessary is that they should fee...,All that is necessary is that they should feel...
8,8,1930-07-28T00:00:00+00:00,""" VIA "" SURAT From the Gujarati original : Bl...",""" VIA "" SURAT From the Gujarati original : Ble..."
9,9,1930-04-19T00:00:00+00:00,From a photostat : S.N. 9569 1 Vande- Pyarela...,From a photostat : S.N. 9569 1 Vande- Pyarelal...
10,10,1930-04-11T00:00:00+00:00,Ganga- behn S.N. 32964 1 LAKSHMIBEHN ( KHARE ...,Ganga- behn S.N. 32964 1 LAKSHMIBEHN ( KHARE )...
11,11,1930-10-12T00:00:00+00:00,LETTER TO CHANDRAKANTA 2 Gangabehn Vaidya 3 F...,LETTER TO CHANDRAKANTA 2 Gangabehn Vaidya 3 Fr...


In [ ]:
import datetime
import pandas as pd
event_data = pd.read_csv('drive/MyDrive/Colab Notebooks/event_timeline_generation/news-tls data/CWAL/letters/CWAL_letters_event_data_new.csv')
response_dic = {}
for index, row in event_data.iterrows():
    try:
        response_dic[datetime.datetime.strptime(row['date'].split("T")[0], "%Y-%m-%d").date()] += str(row['sentence'])
    except:
        response_dic[datetime.datetime.strptime(row['date'].split("T")[0], "%Y-%m-%d").date()] = str(row['sentence'])

response_dic


print(len(response_dic))

!pip install bert-extractive-summarizer
from summarizer import Summarizer
model = Summarizer()

predicted_dic = {}

i=1

for k, events in response_dic.items():
    if i%5 == 0:
        print("processing %s event"%(i))
    i+=1
    res = model(events, num_sentences=2)
    if res != '':
        predicted_dic[k] = res

86


Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


processing 5 event
processing 10 event
processing 15 event
processing 20 event
processing 25 event
processing 30 event
processing 35 event
processing 40 event
processing 45 event
processing 50 event
processing 55 event
processing 60 event
processing 65 event
processing 70 event
processing 75 event
processing 80 event
processing 85 event


In [ ]:
with open('drive/MyDrive/Colab Notebooks/event_timeline_generation/news-tls data/CWMG/json_CWMG_letters_event_data.json', 'w') as content:
    json.dump(predicted_dic, content, indent=4)

In [ ]:
import json
with open('/content/drive/MyDrive/Colab Notebooks/event_timeline_generation/news-tls data/CWAL/letters/timelines.jsonl',  encoding='utf-8') as f:
    jsonl_content = f.read()
    
ground_truths = [json.loads(jline) for jline in jsonl_content.splitlines()]

ground_truth_dic = {}
for gt in ground_truths:
    if gt != []:
        for item in gt:
            # dic[item[0]] = item[1]
            events = ''
            for ev in item[1]:
                events += str(ev)
            ground_truth_dic[datetime.datetime.strptime(item[0].split("T")[0], "%Y-%m-%d").date()] = events

print(ground_truth_dic)


{datetime.date(1848, 1, 1): "['Last year he presided at the convention which nominated the democratic candidate for congress in this District and afterwards ran for the state Senate himself , not desiring the seat , but avowedly to aid and strengthen his party .']", datetime.date(1848, 9, 12): "['Taylor, making his chief theme the fashionable statement of all those who oppose him, the old Locofocos as well as the new that he has no principles, and that the whig party have abandoned their principles by adopting him as their candidate', 'Mr. Lincoln proceeded to rally the Buffalo Convention 4 for forbearing to say anything after all the previous declarations of those members who were formerly Whigs on the subject of the Mexican war , because the Van Burens had been known to have supported it .', 'He declared that of all the parties asking the confidence of the country , this new one had less of principle than any other .', 'He wondered whether it was still the opinion of these Free Soil 

In [ ]:
res_dic = {}


for index, row in event_data.iterrows():
    try:
        res_dic[datetime.datetime.strptime(row['date'].split("T")[0], "%Y-%m-%d").date()].append(str(row['event']))
    except:
        res_dic[datetime.datetime.strptime(row['date'].split("T")[0], "%Y-%m-%d").date()] = [str(row['event'])]

response_dic = {}

for date, events in res_dic.items():
    response_dic[date] = " ".join(events)

# for index, row in event_data.iterrows():
#     try:
#         response_dic[datetime.datetime.strptime(row['date'].split("T")[0], "%Y-%m-%d").date()].append(str(row['event']))
#     except:
#         response_dic[datetime.datetime.strptime(row['date'].split("T")[0], "%Y-%m-%d").date()] = str(row['event'])


print(response_dic)

{datetime.date(1848, 12, 24): ' A. LINCOLN My dear father Washington , Decr . 24th . Your letter of the 7th . While I was at Springfield last fall , Yours truly A Lincoln Yours as ever A. LINCOLN . A. LINCOLN .', datetime.date(1848, 9, 12): ' But , the question of the extension of slavery to new territories of this country , is a part of our responsibility and care , and is under our control . They therefore had taken a position calculated to break down their single important declared object . They were working for the election of either Gen. Cass or Gen. Taylor . Mr. Kellogg 2 then introduced to the meeting the Hon . Mr. Lincoln has a very tall and thin figure , with an intellectual face , showing a searching mind , and a cool judgment . But since there was no fixed law on the subject , and since the whole probable result of their action would be an assistance in electing Gen. CASS , he must say that they were behind the Whigs in their advocacy of the freedom of the soil . He had neve

In [ ]:
new_res_dic = {}

start = list(ground_truth_dic)[0]
end = list(ground_truth_dic)[-1]
for k, v in predicted_dic.items():
    if k >= start and k <= end:
        new_res_dic[k] = v 

new_res_dic

{datetime.date(1848, 1, 1): 'Last year he presided at the convention which nominated the democratic candidate for congress in this District and afterwards ran for the state Senate himself , not desiring the seat , but avowedly to aid and strengthen his party .',
 datetime.date(1848, 9, 12): 'Mr. Kellogg 2 then introduced to the meeting the Hon .Abram Lincoln , whig member of Congress from Illinois , a representative of free soil .Mr. Lincoln then went into a terse though rapid review of the origin of the Mexican war and the connection of the administration and of General Taylor with it , from which he deduced a strong appeal to the Whigs present to do their duty in the support of General Taylor , and closed with the warmest aspirations for and confidence in a deserved success .At the close of this truly masterly and convincing speech , the audience gave three enthusiastic cheers for Illinois , and three more for the eloquent Whig member from that State .',
 datetime.date(1848, 9, 15): 

In [ ]:
!chmod 777 -R /content/notebooks/pyrouge/


evaluator = rouge.TimelineRougeEvaluator(measures=["rouge_1", "rouge_2"])

# predicted_timeline = timelines.Timeline(
#     {datetime.date(2011, 1, 1): ['At least 21 dead and more than 70 injured after bomb explodes outside Coptic church in Alexandria']}
# )

# groundtruth = timelines.GroundTruth(
#     [
#         timelines.Timeline(
#             {datetime.date(2011, 1, 5): ['At least 21 dead and more than 70 injured after bomb explodes outside Coptic church in Alexandria'],
#              datetime.date(2011, 1, 2): ["Egyptian President Hosni Mubarak 's announcement that he will serve out remaining term immediately rejected by angry crowds"]}
#         )
#     ]
# )
predicted_timeline = timelines.Timeline(predicted_dic)

groundtruth = timelines.GroundTruth(
    [
        timelines.Timeline(ground_truth_dic)
    ]
)

pp = pprint.PrettyPrinter(indent=4)

print("concat")
pp.pprint(evaluator.evaluate_concat(predicted_timeline, groundtruth))
print("")
print("align, date-content costs")
pp.pprint(evaluator.evaluate_align_date_content_costs_many_to_one(predicted_timeline, groundtruth))


concat
{   'rouge_1': {   'f_score': 0.09016393442622951,
                   'precision': 0.04721030042918455,
                   'recall': 1.0},
    'rouge_2': {   'f_score': 0.057851239669421496,
                   'precision': 0.03017241379310345,
                   'recall': 0.7}}

align, date-content costs
{   'rouge_1': {   'f_score': 0.019073309943589606,
                   'precision': 0.010002914836517874,
                   'recall': 0.20459480801500343},
    'rouge_2': {'f_score': 0, 'precision': 0.0, 'recall': 0.0}}


In [ ]:
!perl /usr/local/lib/python3.7/dist-packages/pyrouge/tools/ROUGE-1.5.5/ROUGE-1.5.5.pl

/usr/local/lib/python3.7/dist-packages/pyrouge/tools/ROUGE-1.5.5/ROUGE-1.5.5.pl
         [-a (evaluate all systems)] 
         [-c cf]
         [-d (print per evaluation scores)] 
         [-e ROUGE_EVAL_HOME] 
         [-h (usage)] 
         [-H (detailed usage)] 
         [-b n-bytes|-l n-words] 
         [-m (use Porter stemmer)] 
         [-n max-ngram] 
         [-s (remove stopwords)] 
         [-r number-of-samples (for resampling)] 
         [-2 max-gap-length (if < 0 then no gap length limit)] 
         [-3 <H|HM|HMR|HM1|HMR1|HMR2> (for scoring based on BE)] 
         [-u (include unigram in skip-bigram) default no)] 
         [-U (same as -u but also compute regular skip-bigram)] 
         [-w weight (weighting factor for WLCS)] 
         [-v (verbose)] 
         [-x (do not calculate ROUGE-L)] 
         [-f A|B (scoring formula)] 
         [-p alpha (0 <= alpha <=1)] 
         [-t 0|1|2 (count by token instead of sentence)] 
         [-z <SEE|SPL|ISI|SIMPLE>] 
         <ROUG